<a href="https://colab.research.google.com/github/dave502/PyTorch/blob/main/HW_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Создать Dataset для загрузки данных (используем только числовые данные)
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

Вопросы? в личку @Kinetikm

In [ ]:
# !pip install --upgrade wandb
# !pip install --upgrade git+https://github.com/PytorchLightning/pytorch-lightning.git

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch import Tensor
import pandas as pd
import numpy as np
from collections import namedtuple
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')
from sklearn.preprocessing import LabelEncoder

Mounted at /content/drive


In [2]:
df_path = "/content/drive/MyDrive/MLData/train.csv"

*код загрузки датасета писался для Jupyter, для колаба я уже не стал его переписывать*

In [3]:
# %%capture
# !pip install kaggle
# from kaggle.api.kaggle_api_extended import KaggleApi
# api = KaggleApi()
# api.authenticate()
# api.competition_download_file('avito-demand-prediction', 'train.csv', path='data/')

In [4]:
data = pd.read_csv(df_path)
X_train, X_test = train_test_split(data, test_size = 0.25, random_state=13)
print(f'X_train.shape={X_train.shape} X_test.shape={X_test.shape}')

X_train.shape=(1127568, 18) X_test.shape=(375856, 18)


In [5]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.categorical_features = ["region", "city", "parent_category_name", "category_name"]
        self.numerics_features = data.select_dtypes(include=np.number).columns.tolist()
        
        # категориальные признаки
        label_encoders = {}
        for cat_col in self.categorical_features:
          label_encoders[cat_col] = LabelEncoder()
          data[cat_col] = label_encoders[cat_col].fit_transform(data[cat_col])

        # числовые признаки
        # запоняем пропуски в числовых колонках
        data[self.numerics_features] = data[self.numerics_features].apply(pd.to_numeric, errors='coerce')
        # data[self.numerics_features].fillna(0, inplace=True)

        # разделяем обучающий датасет на фичи и целевую переменную 
        self.X_num_train = torch.FloatTensor(data[self.numerics_features].values)
        self.X_num_train[self.X_num_train!=self.X_num_train]=0 # fillna почему-то не заполяет nans, такой способ более оказался эффективным

        self.X_cat_train = torch.IntTensor(data[self.categorical_features].values)
        self.y_train = torch.FloatTensor(data['deal_probability'])

    def __len__(self):
        return len(self.y_train)

    def __getitem__(self, index):
        return self.X_num_train[index], self.X_cat_train[index], self.y_train[index]

    def embedings_dim(self):
      """
      возвращает количество и размерность эмбеддигов
      """
      cat_dims = [torch.unique(self.X_cat_train[:,i]).size()[0] for i,_ in enumerate(self.categorical_features)]
      embedings_dims = [(x, min(50, (x + 1) // 2)) for x in cat_dims]
      return embedings_dims

    def numeric_dim(self):
      """
      возвращает количество числовых фичей (input size для модели)
      """
      return self.X_num_train.size()[1]


In [6]:
train_ds = Dataset(X_train)

In [7]:
train_loader = torch.utils.data.DataLoader(dataset=train_ds, batch_size=32, num_workers=10, drop_last=True)

код нейронной сети со второго дз с небольшими изменениями состава слоёв

In [8]:
class Perceptron(nn.Module):
    
    # tuple с активационными функциями (tuple для неизменности)
    Activatons = namedtuple('Activatons', 'relu silu leaky_relu sigmoid softmax')
    activations = Activatons(F.relu, F.silu, F.leaky_relu, F.sigmoid, F.softmax)
    
    def __init__(self, input_dim, output_dim, activation='relu', activation_params=[]):
        super(Perceptron, self).__init__()
        #fc - полносвязный слой
        self.fc = nn.Linear(input_dim, output_dim)
        #ac - функция активации
        self.ac = getattr(self.activations, activation)
        #ac_params - параметры функции активации
        self.ac_params = activation_params

    def forward(self, x):
        x = self.fc(x)
        return self.ac(x, *self.ac_params)

In [9]:
class Model(nn.Module):
    def __init__(self, num_dim, emb_dim, hidden_dim, output_dim):
        super(Model, self).__init__()

        # эмбеддинг слой для каждого категориального признака
        self.cat_layers = nn.ModuleList([
            nn.Embedding(x, y) for x, y in emb_dim
        ])

        # слой нормализации для числовых признаков   
        self.num_layers = nn.ModuleList([
            nn.BatchNorm1d(num_dim), 
        ])   

        # расчет суммарной выходной размерности слоёв категориальных и числовых признаков
        total_input = sum([x[1] for x in emb_dim]) +  num_dim                          

        # общие слои
        self.next_layers = nn.ModuleList([
            Perceptron(total_input, 4*hidden_dim, "silu"),
            nn.BatchNorm1d(4*hidden_dim),
            nn.Dropout(0.25),
            Perceptron(4*hidden_dim, 4*hidden_dim, "leaky_relu"),
            nn.BatchNorm1d(4*hidden_dim),
            nn.Dropout(0.25),
            Perceptron(4*hidden_dim, output_dim, "sigmoid"),
        ])
        
        
    def forward(self, x_num, x_cat):

        # эмбеддинг слои
        x = [emb_layer(x_cat[:, i])
           for i, emb_layer in enumerate(self.cat_layers)]
        # объединение выходов всех эмбеддинг слоёв
        x = torch.cat(x, 1)

        # числовые слои
        for layer in self.num_layers:
            x_num = layer.forward(x_num) 

        # объединение выходов слоёв для числовых признаков и для категориальных
        x = torch.cat([x, x_num], 1)     

        # дальше общие скрытые слои
        for layer in self.next_layers:
            x = layer.forward(x) 
        return x 
    
    # def predict(self, x):
    #     for layer in self.layers:
    #         x = layer.forward(x) 
    #     return x

log RMSE

In [10]:
class LogRMSE(torch.nn.modules.loss._Loss):
    def __init__(self, reduction: str = 'none') -> None:
        super(LogRMSE, self).__init__(None, None, reduction)
    
    def forward(self, input: Tensor, target: Tensor) -> Tensor:
        se = torch.pow((target - input), 2)
        mse = torch.mean(se)
        rmse = torch.sqrt(mse)
        logrmse = torch.log(rmse)
        return logrmse


In [11]:
model = Model(train_ds.numeric_dim(), train_ds.embedings_dim(), 200, 1)

optimizers = {'SGD': torch.optim.SGD(model.parameters(), lr=0.01),
              'Adam': torch.optim.Adam(model.parameters(), lr=0.01),
              'RMSProp': torch.optim.RMSprop(model.parameters(), lr=0.01),
}
criterion = LogRMSE()

In [ ]:
for optimizer_name, optimizer in optimizers.items():
  print(f'-------------optimizer {optimizer_name}-------------\n')
  EPOCHS = 5
  for epoch in tqdm(range(EPOCHS)):  
      running_loss = 0.0
      for i, data in enumerate(train_loader, 0):
          inputs_num, inputs_cat, labels = data[0], data[1], data[2]

          # обнуляем градиент
          optimizer.zero_grad()
          outputs = model(inputs_num, inputs_cat)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # выводим статистику о процессе обучения
          running_loss += loss.item()
          if i % 5000 == 0:    # печатаем каждые 5000 mini-batches
              print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 5000))
              running_loss = 0.0
  print('\nTraining is finished!')

-------------optimizer SGD-------------



  0%|          | 0/5 [00:00<?, ?it/s]

[1,     1] loss: -0.000
[1,  5001] loss: -1.369
[1, 10001] loss: -1.383
[1, 15001] loss: -1.384
[1, 20001] loss: -1.386
[1, 25001] loss: -1.389
[1, 30001] loss: -1.387
[1, 35001] loss: -1.385


 20%|██        | 1/5 [04:24<17:39, 264.94s/it]

[2,     1] loss: -0.000
[2,  5001] loss: -1.392
[2, 10001] loss: -1.383
[2, 15001] loss: -1.384
[2, 20001] loss: -1.386
[2, 25001] loss: -1.389
[2, 30001] loss: -1.387
[2, 35001] loss: -1.385


...

[5,     1] loss: -0.000
[5,  5001] loss: -1.386
[5, 10001] loss: -1.378
[5, 15001] loss: -1.380
[5, 20001] loss: -1.382
[5, 25001] loss: -1.383
[5, 30001] loss: -1.337
[5, 35001] loss: -1.275


100%|██████████| 5/5 [35:48<00:00, 429.62s/it]



Training is finished!
-------------optimizer RMSProp-------------



  0%|          | 0/5 [00:00<?, ?it/s]

[1,     1] loss: -0.000
[1,  5001] loss: -1.283


*RMSProp не доработал до конца, но результаты понятны...*

Adam и SGD почти не отличаются по метрике, а RMSProp показал результаты немного хуже. Выводы относятся, конечно же к оптимизаторам "из коробки", без дополнительной настройки.

Модель почему-то не обучается. Возможно, это нормальное поведение, но подозреваю, что где-то в коде баг. Визуально вроде всё ок, а отладка займёт много времени. Уж очень затрантное по времени дз в связи с размером датасета.    